In [1]:
%gui wx
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ

os.environ["DIPY_HOME"] = "/home/jovyan/Data"


#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json
import subprocess

Gtk-Message: 12:51:08.645: Failed to load module "canberra-gtk-module"


In [2]:
################
# Start FSLeyes (very neat tool to visualize MRI data of all sorts) within Python
################
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

12:51:24: Debug: Adding duplicate image handler for 'Windows bitmap file'
12:51:24: Debug: Adding duplicate animation handler for '1' type
12:51:24: Debug: Adding duplicate animation handler for '2' type
12:51:24: Debug: Adding duplicate image handler for 'Windows bitmap file'
12:51:24: Debug: Adding duplicate animation handler for '1' type
12:51:24: Debug: Adding duplicate animation handler for '2' type

(ipykernel_launcher.py:1373): Gtk-CRITICAL **: 12:51:25.545: gtk_window_resize: assertion 'height > 0' failed


In [3]:
# Variables
dataset_id = "ds000171"
bids_root = "./ds000171"  # Directory to store the downloaded dataset
# Path to the 'derivatives' directory for processed data
deriv_root = os.path.join(bids_root, 'derivatives')

# Create a directory for preprocessed data (e.g., 'preprocessed_data')
preproc_root = os.path.join(deriv_root, 'preprocessed_data')

In [4]:


# Create the target directory if it does not exist
if not os.path.exists(bids_root):
    os.makedirs(bids_root)

# Download the raw data from OpenNeuro into the target directory
#openneuro.download(dataset=dataset_id, target_dir= bids_root)




In [10]:
if not os.path.exists(preproc_root):
    os.makedirs(preproc_root)

In [5]:
subjects = [d for d in os.listdir(bids_root) if d.startswith('sub-con')]
# Loop over all subjects and create directories for each subject's processed data
for subject in subjects:
    subject_dir = os.path.join(preproc_root, subject)

    # Create subdirectories for anatomical and functional data
    os.makedirs(os.path.join(subject_dir, 'anat'), exist_ok=True)
    os.makedirs(os.path.join(subject_dir, 'func'), exist_ok=True)

print(f"Directories created for processed data in {preproc_root}")

Directories created for processed data in ./ds000171/derivatives/preprocessed_data


In [6]:

subjects = [d for d in os.listdir(bids_root) if d.startswith('sub-con')]
anatomical_path = op.join(bids_root, subjects[0], 'anat', '{}_T1w.nii.gz'.format(subjects[0]))

# Check if the anatomical file exists before calling BET
if not os.path.exists(anatomical_path):
    print(f"Error: Anatomical file not found: {anatomical_path}")
else : 
    print("oui")

oui


In [17]:
#________________________________ ANATOMICAL PREPROCESSING ____________________________________________
subjects = [d for d in os.listdir(bids_root) if d.startswith('sub-con')]
def get_skull_stripped_anatomical(bids_root, preproc_root, subject, robust=False):
    """
    Function to perform skull-stripping (removing the skull around the brain).
    This is a simple wrapper around the brain extraction tool (BET) in FSL's suite
    It assumes data to be in the BIDS format (which we will cover in the following weeks).
    The method also saves the brain mask which was used to extract the brain.

    The brain extraction is conducted only on the T1w of the participant.

    Parameters
    ----------
    bids_root: string
        The root of the BIDS directory
    preproc_root: string
        The root of the preprocessed data, where the result of the brain extraction will be saved.
    subject_id: string
        Subject ID, the subject on which brain extraction should be conducted.
    robust: bool
        Whether to conduct robust center estimation with BET or not. Default is False.
    """
    anatomical_path = op.join(bids_root, subject, 'anat', '{}_T1w.nii.gz'.format(subject))
    betted_brain_path = op.join(preproc_root, subject, 'anat', '{}_T1w'.format(subject))
    os.system('bet {} {} -m {}'.format(anatomical_path, betted_brain_path, '-R' if robust else ''))




#for sub in tqdm(subjects) : 
    #get_skull_stripped_anatomical(bids_root, preproc_root, sub, robust = True)

#print("Done with BET.")


In [19]:
subject_01 = 'sub-control01'

In [20]:
get_skull_stripped_anatomical(bids_root, preproc_root, subject_01, robust=True)

In [22]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.join(preproc_root, subject_01, 'anat', '{}_T1w.nii.gz'.format(subject_01)))
fsleyesDisplay.load(resulting_mask_path)

In [24]:
#______________________TISSUE SEGMENTATION_______________

anatomical_path = op.join(bids_root, subject_01, 'anat', '{}_T1w.nii.gz'.format(subject_01))
bet_path = op.join(preproc_root, subject_01, 'anat', '{}_T1w'.format(subject_01))

#########################
# Solution
# By reading above: we must apply FAST to the brain-extracted image. Thus we must use the BET path brain.
##########################
fast_target = bet_path # Replace with either anatomical_path or bet_path (note: you can try both and decide which is more reasonable!)

[os.remove(f) for f in glob.glob(op.join(preproc_root, subject_01, 'anat', '*fast*'))] # Just to clean the directory in between runs of the cell
segmentation_path = op.join(preproc_root, subject_01, 'anat', '{}_T1w_fast'.format(subject_01))
fast(imgs=[fast_target], out=segmentation_path, n_classes=3)

{}

In [25]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(bet_path)
fsleyesDisplay.load(glob.glob(op.join(preproc_root, subject_01, 'anat','*pve_0*'))[0])
fsleyesDisplay.load(glob.glob(op.join(preproc_root, subject_01, 'anat','*pve_1*'))[0])
fsleyesDisplay.load(glob.glob(op.join(preproc_root, subject_01, 'anat','*pve_2*'))[0])
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[1]).cmap = 'Red'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[2]).cmap = 'Green'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[3]).cmap = 'Blue'

In [28]:
#________________COREGISTRATION_____________________
from fsl.wrappers import flirt


subject_anatomical = op.join(preproc_root, subject_01, 'anat', '{}_T1w'.format(subject_01))
mni_template = op.expandvars(op.join('$FSLDIR', 'data', 'standard', 'MNI152_T1_1mm_brain'))

###################
# Select which image should be target or reference
# ANSWER:
# The subject anatomical will most often be the target, as the template is usually where we want to map our subjects for 
# group comparison.
# There are cases, however, where we may want the subject anatomical to be the reference. This is the case when we want to map an 
# atlas to a subject while preserving the subject as much as possible.
# We showcase here the case where the subject is chosen as target.
##################
target = subject_anatomical # Fill me
reference = mni_template # Fill me
result = op.join(preproc_root, subject_01, 'anat', '{}_T1w_mni'.format(subject_01))
flirt(target, reference, out=result)



Final result: 
0.003348 0.005721 -1.116064 204.794610 
-0.933350 0.500887 0.005750 178.784833 
0.501418 1.048657 -0.003251 -149.700516 
0.000000 0.000000 0.000000 1.000000 



{}

In [29]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(reference) 
fsleyesDisplay.load(result)